In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
import evaluate


C:\Users\aarav\anaconda3\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\aarav\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
C:\Users\aarav\anaconda3\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issu

In [2]:
# Load dataset
df = pd.read_csv('intent_dataset.csv')

# Prepare labels
le = LabelEncoder()
df['subintent_label'] = le.fit_transform(df['sub_intent'])

# Split data (80% train, 20% test)
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['sentence'].tolist(),
    df['subintent_label'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df['subintent_label']
)

print(f"Training samples: {len(train_texts)}")
print(f"Testing samples: {len(test_texts)}")
print(f"Label classes: {le.classes_}")


Training samples: 219
Testing samples: 55
Label classes: ['account_password_reset' 'account_statement' 'account_update_mobile'
 'bill_pay_issue' 'budget_planning' 'card_activation' 'card_lost_blocked'
 'card_replacement' 'expense_tracking' 'freeze_account'
 'get_security_advice' 'goodbye_general' 'greeting_general'
 'insurance_claim_help' 'insurance_policy_inquiry'
 'insurance_premium_query' 'investment_returns' 'loan_apply_steps'
 'loan_eligibility_check' 'loan_interest_info' 'loan_statement'
 'mutual_funds_inquiry' 'refund_status' 'report_suspicious_activity'
 'saving_tips' 'stock_market_query' 'upi_payment_failure']


In [3]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=32)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=32)


In [4]:
class IntentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)

train_dataset = IntentDataset(train_encodings, train_labels)
test_dataset = IntentDataset(test_encodings, test_labels)


In [5]:
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=len(le.classes_)
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
    logging_steps=10,
    disable_tqdm=False
)

accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


In [7]:
trainer.train()


Step,Training Loss
10,3.297300
20,3.303900
30,3.214200
40,3.075700
50,2.995100
60,2.844500
70,2.702200
80,2.692400


TrainOutput(global_step=84, training_loss=3.0034325349898565, metrics={'train_runtime': 43.4377, 'train_samples_per_second': 15.125, 'train_steps_per_second': 1.934, 'total_flos': 2550875472990.0, 'train_loss': 3.0034325349898565, 'epoch': 3.0})

In [8]:
results = trainer.evaluate()
print("Evaluation results:", results)


Evaluation results: {'eval_loss': 2.7165229320526123, 'eval_accuracy': 0.6, 'eval_runtime': 0.6764, 'eval_samples_per_second': 81.315, 'eval_steps_per_second': 10.349, 'epoch': 3.0}


In [9]:
model.save_pretrained('./model', safe_serialization=False)
tokenizer.save_pretrained('./model')

print("✅ Model saved!")
print("✅ Tokenizer saved!")


✅ Model saved!
✅ Tokenizer saved!


In [10]:
model = DistilBertForSequenceClassification.from_pretrained('./model')
tokenizer = DistilBertTokenizerFast.from_pretrained('./model')


In [11]:
def predict_intent(text):
    inputs = tokenizer(text, truncation=True, padding=True, max_length=32, return_tensors="pt")
    outputs = model(**inputs)
    pred_label_id = torch.argmax(outputs.logits, dim=-1).item()
    intent = le.inverse_transform([pred_label_id])[0]
    return intent


In [12]:
def detect_bank(text):
    """Detects which bank/platform user is referring to"""
    text_lower = text.lower()
    
    if 'sbi' in text_lower or 'state bank' in text_lower:
        return 'SBI'
    elif 'hdfc' in text_lower:
        return 'HDFC'
    elif 'icici' in text_lower:
        return 'ICICI'
    elif 'axis' in text_lower:
        return 'Axis'
    elif 'kotak' in text_lower:
        return 'Kotak'
    elif 'google pay' in text_lower or 'gpay' in text_lower:
        return 'Google Pay'
    elif 'paytm' in text_lower:
        return 'Paytm'
    elif 'phonepe' in text_lower or 'phone pe' in text_lower:
        return 'PhonePe'
    else:
        return None


In [13]:
INTENT_HANDLERS = {
    'account_password_reset': {
        'SBI': {
            'message': "I can help you reset your SBI password. Which service do you need help with?",
            'workflows': [
                {
                    'name': 'Internet Banking (OnlineSBI)',
                    'steps': [
                        "Visit https://retail.onlinesbi.sbi/retail/login.htm",
                        "Click 'Forgot Login Password'",
                        "Enter your Username",
                        "Enter OTP sent to registered mobile",
                        "Create new password"
                    ],
                    'link': 'https://retail.onlinesbi.sbi/retail/login.htm'
                },
                {
                    'name': 'YONO App',
                    'steps': [
                        "Open YONO SBI app",
                        "Tap 'Forgot Password'",
                        "Enter CIF/Username",
                        "Verify via OTP",
                        "Set new password"
                    ]
                }
            ]
        },
        'HDFC': {
            'message': "I can help you reset your HDFC password.",
            'workflows': [
                {
                    'name': 'NetBanking',
                    'steps': [
                        "Go to https://netbanking.hdfcbank.com",
                        "Click 'Forgot IPIN'",
                        "Enter Customer ID",
                        "Verify via Debit Card",
                        "Enter OTP",
                        "Create new password"
                    ],
                    'link': 'https://netbanking.hdfcbank.com'
                }
            ]
        },
        'ICICI': {
            'message': "To reset your ICICI password:",
            'workflows': [
                {
                    'name': 'Internet Banking',
                    'steps': [
                        "Visit https://infinity.icicibank.com",
                        "Click 'Forgot User ID/Password'",
                        "Enter registered mobile/email",
                        "Verify OTP",
                        "Create new password"
                    ],
                    'link': 'https://infinity.icicibank.com'
                },
                {
                    'name': 'iMobile App',
                    'steps': [
                        "Open iMobile app",
                        "Tap 'Forgot Password'",
                        "Enter registered mobile",
                        "Verify OTP",
                        "Set new password"
                    ]
                }
            ]
        },
        'Axis': {
            'message': "To reset your Axis Bank password:",
            'workflows': [
                {
                    'name': 'Internet Banking',
                    'steps': [
                        "Go to https://retail.axisbank.co.in",
                        "Click 'Forgot Password'",
                        "Enter Customer ID",
                        "Verify via registered mobile",
                        "Create new password"
                    ],
                    'link': 'https://retail.axisbank.co.in'
                }
            ]
        },
        'Kotak': {
            'message': "To reset your Kotak Mahindra password:",
            'workflows': [
                {
                    'name': 'Net Banking',
                    'steps': [
                        "Visit https://netbanking.kotak.com",
                        "Click 'Forgot Password'",
                        "Enter CRN/Customer ID",
                        "Verify via OTP",
                        "Set new password"
                    ],
                    'link': 'https://netbanking.kotak.com'
                }
            ]
        },
        'Google Pay': {
            'message': "To reset your Google Pay PIN:",
            'workflows': [
                {
                    'name': 'Reset Google Pay PIN',
                    'steps': [
                        "Open Google Pay app",
                        "Tap profile picture (top right)",
                        "Go to Settings → Privacy & Security",
                        "Tap 'Change Google Pay PIN'",
                        "Verify identity (fingerprint/screen lock)",
                        "Enter new 4-6 digit PIN"
                    ]
                }
            ]
        },
        'Paytm': {
            'message': "To reset your Paytm password:",
            'workflows': [
                {
                    'name': 'Reset Password',
                    'steps': [
                        "Open Paytm app",
                        "Tap Profile → Settings",
                        "Select 'Change Password'",
                        "Verify via OTP",
                        "Enter new password"
                    ]
                }
            ]
        },
        'PhonePe': {
            'message': "To reset your PhonePe PIN:",
            'workflows': [
                {
                    'name': 'Reset PIN',
                    'steps': [
                        "Open PhonePe app",
                        "Tap Profile icon",
                        "Go to Settings",
                        "Select 'Change PIN'",
                        "Verify via OTP",
                        "Enter new PIN"
                    ]
                }
            ]
        }
    },
    
    'account_statement': {
        'SBI': {
            'message': "Here's how to get your SBI account statement:",
            'workflows': [
                {
                    'name': 'YONO App',
                    'steps': [
                        "Login to YONO SBI",
                        "Go to Accounts → Select account",
                        "Tap Statement",
                        "Select date range",
                        "Download PDF"
                    ]
                },
                {
                    'name': 'Internet Banking',
                    'steps': [
                        "Login to OnlineSBI",
                        "Go to Account Statement",
                        "Select account and date range",
                        "Download statement"
                    ],
                    'link': 'https://retail.onlinesbi.sbi'
                }
            ]
        },
        'HDFC': {
            'message': "To get your HDFC account statement:",
            'workflows': [
                {
                    'name': 'NetBanking',
                    'steps': [
                        "Login to HDFC NetBanking",
                        "Go to Accounts → Statement",
                        "Select date range",
                        "Download PDF"
                    ]
                }
            ]
        },
        'ICICI': {
            'message': "To get your ICICI account statement:",
            'workflows': [
                {
                    'name': 'Internet Banking',
                    'steps': [
                        "Login to ICICI NetBanking",
                        "Go to Accounts → Statement",
                        "Select account and date range",
                        "Download statement"
                    ]
                },
                {
                    'name': 'iMobile App',
                    'steps': [
                        "Open iMobile app",
                        "Tap Accounts",
                        "Select Statement",
                        "Download or email"
                    ]
                }
            ]
        },
        'Axis': {
            'message': "To get your Axis account statement:",
            'workflows': [
                {
                    'name': 'Internet Banking',
                    'steps': [
                        "Login to Axis NetBanking",
                        "Go to Accounts → Statement",
                        "Download statement"
                    ]
                }
            ]
        }
    },
    
    'card_lost_blocked': {
        'SBI': {
            'message': "⚠️ URGENT: Block your SBI card immediately:",
            'workflows': [
                {
                    'name': 'Customer Care (24x7)',
                    'steps': [
                        "Call: 1800 11 2211 or 1800 425 3800",
                        "Select Block Card option",
                        "Provide card details",
                        "Card blocked instantly"
                    ],
                    'urgent': True
                }
            ]
        },
        'HDFC': {
            'message': "⚠️ Block your HDFC card immediately:",
            'workflows': [
                {
                    'name': 'PhoneBanking',
                    'steps': [
                        "Call: 1800 266 4332",
                        "Request card blocking",
                        "Card blocked immediately"
                    ],
                    'urgent': True
                }
            ]
        },
        'ICICI': {
            'message': "⚠️ Block your ICICI card immediately:",
            'workflows': [
                {
                    'name': 'Customer Care',
                    'steps': [
                        "Call: 1860 120 7777",
                        "Request card blocking",
                        "Verify identity",
                        "Card blocked"
                    ],
                    'urgent': True
                }
            ]
        }
    },
    
    'upi_payment_failure': {
        'Google Pay': {
            'message': "If your Google Pay payment failed:",
            'workflows': [
                {
                    'name': 'Check Payment Status',
                    'steps': [
                        "Open Google Pay",
                        "Tap Activity/Transactions",
                        "Find failed transaction",
                        "Auto-refund in 5-7 days if money deducted",
                        "Tap transaction → Get Help for disputes"
                    ]
                }
            ]
        },
        'Paytm': {
            'message': "For Paytm payment failure:",
            'workflows': [
                {
                    'name': 'Check and Resolve',
                    'steps': [
                        "Open Paytm",
                        "Go to Passbook",
                        "Find failed transaction",
                        "Tap → Raise Issue",
                        "Refund in 7 working days"
                    ]
                }
            ]
        },
        'PhonePe': {
            'message': "For PhonePe payment failure:",
            'workflows': [
                {
                    'name': 'Resolve Failure',
                    'steps': [
                        "Open PhonePe",
                        "Go to History",
                        "Find failed payment",
                        "Tap Report Issue",
                        "Refund in 5-7 days"
                    ]
                }
            ]
        }
    }
}


In [14]:
def handle_user_query_enhanced(user_input):
    """Complete workflow: Intent + Bank + Response"""
    
    # Step 1: Predict intent
    predicted_intent = predict_intent(user_input)
    
    # Step 2: Detect bank
    bank = detect_bank(user_input)
    
    # Step 3: Build response
    response = {
        'user_query': user_input,
        'detected_intent': predicted_intent,
        'detected_bank': bank,
        'response': None
    }
    
    # Check if we have handler for this intent
    if predicted_intent in INTENT_HANDLERS:
        intent_data = INTENT_HANDLERS[predicted_intent]
        
        # If bank detected and we have data for it
        if bank and bank in intent_data:
            response['response'] = intent_data[bank]
            response['response']['type'] = 'workflow'
        else:
            # Ask which bank
            response['response'] = {
                'message': f"I can help you with {predicted_intent.replace('_', ' ')}. Which bank/platform are you using?",
                'available_banks': list(intent_data.keys()),
                'type': 'bank_selection'
            }
    else:
        response['response'] = {
            'message': f"I detected your query is about '{predicted_intent.replace('_', ' ')}', but I don't have detailed guidance for this yet. Could you please rephrase or specify your bank?",
            'type': 'not_found'
        }
    
    return response


In [15]:
def display_response(response_dict):
    message = response_dict.get('response', {}).get('message', '')
    workflows = response_dict.get('response', {}).get('workflows', [])
    available_banks = response_dict.get('response', {}).get('available_banks', [])
    
    print(f"Advisor: {message}")
    
    if available_banks:
        print(f"📌 Available: {', '.join(available_banks)}")
    
    if workflows:
        print("="*60)
        for workflow in workflows:
            print(f"\n📋 {workflow['name']}")
            print("-"*60)
            for i, step in enumerate(workflow['steps'], 1):
                print(f"  {i}. {step}")
            if workflow.get('link'):
                print(f"  🔗 {workflow['link']}")
        print("="*60)
    print()


In [16]:
def financial_advisor_chatbot(user_input):
    return handle_user_query_enhanced(user_input)


In [17]:
context_pending_bank = None
context_pending_intent = None

while True:
    user_query = input("User: ")
    if user_query.lower() in ("exit", "quit", "q"):
        print("Exiting test.")
        break

    # If bot is waiting for the bank reply after original question
    if context_pending_bank is not None and context_pending_intent is not None:
        # Instead of combining queries, use the original detected intent and the replied bank
        # Create a fake input string just for bank detection
        detected_bank = detect_bank(user_query)
        response = {
            'user_query': context_pending_bank,
            'detected_intent': context_pending_intent,
            'detected_bank': detected_bank,
            'response': None
        }
        # Look up in INTENT_HANDLERS directly, bypass intent prediction
        handler = INTENT_HANDLERS.get(context_pending_intent)
        if detected_bank and handler and detected_bank in handler:
            response['response'] = handler[detected_bank]
            response['response']['type'] = 'workflow'
        else:
            # Fallback if bank still not recognized
            response['response'] = {
                'message': f"I can help you with {context_pending_intent.replace('_', ' ')}. Which bank/platform are you using?",
                'available_banks': list(handler.keys()) if handler else [],
                'type': 'bank_selection'
            }
        display_response(response)
        context_pending_bank = None
        context_pending_intent = None
    else:
        # Normal case: full prediction pipeline
        response = handle_user_query_enhanced(user_query)
        display_response(response)
        # If bot asks for bank, save context
        if response.get('response', {}).get('type') == 'bank_selection':
            context_pending_bank = user_query
            context_pending_intent = response.get('detected_intent')


User: q
Exiting test.
